## Introduction

## Data Preperation

### Load Data from Github

The code below uses the requests library to fetch contnets of the projects repository and fetches each text document in the sub directories 'pos' and 'neg'. The documents fetched are used to create a Dataframe with all reviews and the type of the review (as either 'neg' or 'pos')

In [38]:
import requests
import pandas as pd

base_url = "https://api.github.com/repos/ktxdev/AIM-5001/contents/"

def create_reviews_dataframe(df, base_url, path):
    url = f"{base_url}{path}"
    response = requests.get(url)

    # Process response if request was successful
    if response.status_code == 200:
        # Store json content from the request
        contents = response.json()

        # Iterate over each item in the response's json content
        for item in contents:
            # Validate the type of item 
            if item['type'] == 'file':
                # Get the file 
                review = requests.get(item['download_url'])
                # Get text contained in the file
                text = review.text
                # Get type of review from path
                type = 'pos' if path.endswith('pos') else 'neg'
                # Create a dataframe for the current file
                df2 = pd.DataFrame({'text': [text], 'type': [type]})
                # Append the dataframe to the main dataframe
                df = pd.concat([df, df2])
            elif item['type'] == 'dir':
                # Recursive call 
                return create_reviews_dataframe(df, base_url, item['path'])
    else:
        print(f"Failed to fetch directory contents with status code: {response.status_code}")
    
    return df

reviews_df = create_reviews_dataframe(pd.DataFrame(columns=['text', 'type']), base_url, "M12/Assn/1.%20Data")
reviews_df.head()

,text,type
0,"plot : two teen couples go to a church party ,...",neg
0,the happy bastard's quick movie review \ndamn ...,neg
0,it is movies like these that make a jaded movi...,neg
0,""" quest for camelot "" is warner bros . ' firs...",neg
0,synopsis : a mentally unstable man undergoing ...,neg


### Preprocessing the data

In this step I will preprocess the data by converting it to lower case, removing all panctuations and stop words

In [41]:
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

english_stopwords = stopwords.words('english')

# Converting text to lowercase
reviews_df['text'] = reviews_df['text'].str.lower()
# Remove panctuations
reviews_df['text'] = reviews_df['text'].apply(lambda x: ' '.join(x for x in x.split() if x not in string.punctuation))
# Remove stopwords
reviews_df['text'] = reviews_df['text'].apply(lambda x: ' '.join(x for x in x.split() if x not in english_stopwords))
# Lemmatize the words
lemmatizer = WordNetLemmatizer()
reviews_df['text'] = reviews_df['text'].apply(lambda x: ' '.join(lemmatizer.lemmatize(x) for x in x.split()))
reviews_df.head()


,text,type
0,plot two teen couple go church party drink dri...,neg
0,happy bastard's quick movie review damn y2k bu...,neg
0,movie like make jaded movie viewer thankful in...,neg
0,quest camelot warner bros first feature-length...,neg
0,synopsis mentally unstable man undergoing psyc...,neg


### Creating the Term-Document Matrix

In [46]:
set()

TypeError: unhashable type: 'list'